In [10]:
import asyncio
import aioodbc
import pyodbc
import pandas as pd

def create_connection():
    connection_string = (
        'DRIVER={FreeTDS};'
        'SERVER=192.168.0.20;'
        'PORT=1433;'
        'DATABASE=expreso;'
        'UID=datalitica;'
        'PWD=425am953;'
        'TDS_Version=7.2;'
    )
    try:
        conn = pyodbc.connect(connection_string)
        return conn
    except Exception as e:
        print(f"Error al conectar: {e}")
        return None


# def get_table_data_as_df(conn, table_name):
#     query = f"SELECT * FROM {table_name}"
#     with conn.cursor() as cursor:
#         cursor.execute(query)
        
#         # Obtener columnas de manera segura
#         columns = []
#         if cursor.description:
#             columns = [column[0] for column in cursor.description]
#         else:
#             print(f"Advertencia: No se detectaron columnas en {table_name}")
#             return pd.DataFrame()
            
#         data = cursor.fetchall()
        
#         if not data:
#             print(f"Tabla vacía: {table_name}")
#             return pd.DataFrame(columns=columns)
            
#         # Verificación mejorada de consistencia
#         try:
#             return pd.DataFrame.from_records(data, columns=columns)
#         except ValueError as e:
#             print(f"Error en estructura de {table_name}: {str(e)}")
#             # Si falla, devolver datos crudos en una columna
#             return pd.DataFrame(data, columns=['datos_crudos'])
        
# def get_table_data_as_df(conn, table_name):
#     print (f"{table_name}")
#     query = f"SELECT * FROM {table_name}"
#     try:
#         # Usar read_sql para obtener directamente el DataFrame
#         df = pd.read_sql(query, conn)
#         if df.empty:
#             print(f"Tabla vacía: {table_name}")
#         return df
#     except Exception as e:
#         print(f"Error al leer {table_name}: {str(e)}")
#         # Si falla, devolver DataFrame vacío
#         return pd.DataFrame()


def get_table_data_as_df(conn, table_name):
    print(f"Leyendo tabla: {table_name}")
    query = f"SELECT * FROM {table_name}"
    try:
        # Usar read_sql con chunksize para manejar grandes volúmenes de datos
        chunks = pd.read_sql(query, conn, chunksize=10000)
        df = pd.concat(chunks, ignore_index=True)
        
        if df.empty:
            print(f"Tabla vacía: {table_name}")
        return df
    except Exception as e:
        print(f"Error al leer {table_name}: {str(e)}")
        return pd.DataFrame()

# ... existing code ...

In [3]:
import psycopg2

# Datos de conexión
DB_HOST = "localhost"
DB_PORT = "5432"  # Puerto por defecto de Postgres
DB_NAME = "rivadavia"
DB_USER = "rivadavia_user"
DB_PASSWORD = "Iud1g8Rou!JV&n"

try:
    # Crear la conexión
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    
    # Crear un cursor
    cur = conn.cursor()
    
    # Ejecutar una consulta
    cur.execute("SELECT version();")
    
    # Obtener resultado
    version = cur.fetchone()
    print("Conectado a:", version)
    
    # Cerrar cursor y conexión
    cur.close()
    conn.close()

except Exception as e:
    print("Error al conectar:", e)


Conectado a: ('PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit',)


In [18]:
from sqlalchemy import create_engine
connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

def save_dataframe_to_postgres(df, table_name, schema, if_exists='replace'):
    """
    Guarda un DataFrame en una tabla de PostgreSQL
    
    Parámetros:
    df: DataFrame a guardar
    table_name: Nombre de la tabla
    if_exists: Comportamiento si la tabla existe ('replace', 'append', 'fail')
    """
    try:
        # Guardar el DataFrame en PostgreSQL
        df.to_sql(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists=if_exists,
            index=False,
            method='multi',  # Inserción más eficiente
            chunksize=10000  # Insertar en bloques
        )
        print(f"Tabla {schema}.{table_name} creada/actualizada correctamente")
    except Exception as e:
        print(f"Error al guardar {table_name}: {str(e)}")

In [ ]:
save_dataframe_to_postgres(df_asientos, 'asientos', 'expreso')
save_dataframe_to_postgres(df_usuarios, 'usuarios', 'expreso')
save_dataframe_to_postgres(df_guias, 'guias', 'expreso')
save_dataframe_to_postgres(df_guiatraza, 'guiatraza', 'expreso')
save_dataframe_to_postgres(df_ticket, 'ticket', 'expreso')
save_dataframe_to_postgres(df_clientes, 'clientes', 'expreso')
save_dataframe_to_postgres(df_ubicacionguia, 'ubicacionguia', 'expreso')
save_dataframe_to_postgres(df_valores, 'valores', 'expreso')
save_dataframe_to_postgres(df_resumen, 'resumen', 'expreso')
save_dataframe_to_postgres(df_resumen1, 'resumen1', 'expreso')
save_dataframe_to_postgres(df_sucursal, 'sucursal', 'expreso')
save_dataframe_to_postgres(df_saldos, 'saldos', 'expreso')

Error al guardar asientos: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)
Tabla expreso.usuarios creada/actualizada correctamente


In [11]:
conn = create_connection()

In [12]:
df_asientos = get_table_data_as_df(conn, 'Asientos')
df_usuarios = get_table_data_as_df(conn, 'Usuarios')
df_guias = get_table_data_as_df(conn, 'Guias')
df_guiatraza = get_table_data_as_df(conn, 'GuiaTraza')
df_ticket = get_table_data_as_df(conn, 'Ticket')
df_clientes = get_table_data_as_df(conn, 'Clientes')
df_ubicacionguia = get_table_data_as_df(conn, 'UbicacionGuia')
df_valores = get_table_data_as_df(conn, 'Valores')
df_resumen = get_table_data_as_df(conn, 'Resumen')
df_resumen1 = get_table_data_as_df(conn, 'Resumen1')
df_sucursal = get_table_data_as_df(conn, 'Sucursal')
df_saldos = get_table_data_as_df(conn, 'Saldos')

Leyendo tabla: Asientos


/tmp/ipykernel_127347/3060959141.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunks = pd.read_sql(query, conn, chunksize=10000)


Leyendo tabla: Usuarios
Leyendo tabla: Guias


/tmp/ipykernel_127347/3060959141.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_127347/3060959141.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(chunks, ignore_index=True)


Leyendo tabla: GuiaTraza


/tmp/ipykernel_127347/3060959141.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunks = pd.read_sql(query, conn, chunksize=10000)


Leyendo tabla: Ticket
Leyendo tabla: Clientes
Leyendo tabla: UbicacionGuia
Leyendo tabla: Valores
Leyendo tabla: Resumen
Leyendo tabla: Resumen1
Leyendo tabla: Sucursal
Leyendo tabla: Saldos


In [14]:
df_guias

,GuiTipo,GuiNumsuc,GuiNumero,GuiOri,GuiFecha,GuiCli,GuiCliRem,GuiRem,GuiRemDom,GuiCliDes,...,GuiRemitoSuc,GuiFactura,GuiFacturaSuc,GuiSacSucId,GuiTipoDocDes,GuiTipoDocRem,GUiFajo,GuiCaja,GuiSucIdPag,GuiContado
0,A,1,1540,1,2006-07-25,4081,4081,FLORENSA ARGENTINA SA,LAPRIDA 125,4081,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,None
1,A,1,1628,3,2006-12-06,0,0,MARKETING AGRICOLA SRL.,AV.JAPON 455,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,None
2,A,1,14662,1,2006-02-20,0,0,CARDOZO,,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,None
3,A,1,14809,11,2006-02-22,0,0,ELECTROMAT,,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,None
4,A,1,14913,1,2006-02-20,0,0,INST,,0,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934218,X,57,74,9,2024-12-20,0,0,RODRIGO TULA,,13991,...,0.0,0.0,0.0,0.0,80.0,99.0,0,0,10,S
2934219,X,57,75,9,2024-12-24,14012,4769,SUC. DE CASTIÑEIRA MARIA YGNAC,ALEM 670 (LA BANDA),14012,...,35.0,0.0,0.0,1.0,80.0,80.0,0,0,19,N
2934220,X,57,76,9,2024-12-26,14068,14068,DIAZ CLAUDIO RUBEN,UNZAGA SUR 140 - B° PARQUE,0,...,0.0,0.0,0.0,102.0,99.0,96.0,0,0,9,N
2934221,X,57,77,1,2025-01-16,12350,12350,RAMA SRL,MARCOS PAZ 1721,0,...,0.0,0.0,0.0,0.0,99.0,80.0,0,0,9,N


In [3]:
tables = [
    'Asientos',
    'Usuarios',
    'Guias',
    'GuiaTraza',
    'Ticket',
    'Clientes',
    'UbicacionGuia',
    'Valores',
    'Resumen',
    'Resumen1',
    'Sucursal',
    'Saldos',]

conn = create_connection()
if conn:
    dataframes = {}  # Diccionario para almacenar DataFrames
    for i, table in enumerate(tables):
        print(f"{i}. {table}")
        dataframes[f'df_{table}'] = get_table_data_as_df(conn, table)
    conn.close()

0. Asientos


/tmp/ipykernel_85329/1937224564.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


1. Usuarios
2. Guias


: 

In [2]:
dataframes

NameError: name 'dataframes' is not defined